In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Sioux

In [2]:
numNodes = Compute_Jacobian_Sioux.numNodes;
numLinks = Compute_Jacobian_Sioux.numLinks;
numZones = Compute_Jacobian_Sioux.numZones;
numODpairs = Compute_Jacobian_Sioux.numODpairs;
od_pairs = Compute_Jacobian_Sioux.od_pairs;
link_list_js = Compute_Jacobian_Sioux.link_list_js;
link_length_list = Compute_Jacobian_Sioux.link_length_list;

In [3]:
include("../Julia_files/initia_data.jl");
include("../Julia_files/load_network_uni_class.jl");
include("prepare_data_Sioux.jl");
include("extract_data_Sioux.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

In [4]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [5]:
fcoeffs = [1, 0, 0, 0, .15];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [6]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[0]);

In [7]:
deg = 6
c = 3.5
lam = 1.0

coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDict[1], arcsDict[0]);

In [8]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], coeffs_dict[(deg, c, lam, 1)]);

demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

In [9]:
# weight parameters
gamma1 = 0
gamma2 = 1

1

In [10]:
demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [11]:
# get arcs data corresponding to initial demands and flows
arcsDict[1] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[1]);

In [12]:
linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [13]:
linkCostDicDict[1]["0"], link_length_list[1]

(6.028714025602726,6.0)

In [14]:
jacobiSpiessDict[1] = Compute_Jacobian_Sioux.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

In [15]:
# maximum number of iterations
N = 100;  

# Armijo rule parameters
rho = 2;
M = 10;

# search direction parameter
epsilon_1 = 0;

# stop criterion parameter
epsilon_2 = 1e-20;

In [16]:
for l = 1:N
    
    jacobDict[l] = jacobiSpiessDict[l]
    
    demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

    descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[1], tapFlowVecDict[l], 
        tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

    searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

    thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

    demandsVecDict[l+1] = similar(demandsVecDict[0]);
    
    demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
        demandsVecDict[1], coeffs_dict[(deg, c, lam, l)], searchDirecDict[l], thetaMaxDict[l], rho, M);
            
    demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);
    
    tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], coeffs_dict[(deg, c, lam, l)]);
        
    arcsDict[l+1] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[l+1]);
        
    coeffs_dict[(deg, c, lam, l+1)], objInvVIDict[l+1] = train(lam, deg, c, demandsDict[l+1], arcsDict[0]);
    
#     objFunTmp = objF(gamma1, gamma2, demandsVecDict[l+1], demandsVecDict[1], 
#         coeffs_dict[(deg, c, lam, l+1)])
    
#     if objFunTmp > objFunDict[l+1]
#         coeffs_dict[(deg, c, lam, l+1)] = coeffs_dict[(deg, c, lam, l)]
#     end

    linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], coeffs_dict[(deg, c, lam, l+1)]);
    
    jacobiSpiessDict[l+1] = Compute_Jacobian_Sioux.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);
        
    demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);
    
    # stopping criterion
    if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
        break
    end
        
end

# normalize objective function value
for l = 1:(length(objFunDict))
    norObjFunDict[l] = objFunDict[l] / objFunDict[1];
end

In [17]:
demandsVecDict

Dict{Any,Any} with 10 entries:
  0 => [100.0,100.0,500.0,200.0,300.0,500.0,800.0,500.0,1300.0,500.0  …  400.0,…
  4 => [95.744,110.266,447.263,217.68,250.992,544.221,874.041,431.452,1445.4,51…
  7 => [80.7667,114.844,453.894,215.346,234.188,539.259,866.207,416.629,1425.67…
  9 => [79.231,114.912,453.665,214.252,232.39,537.505,864.907,414.169,1422.04,5…
  2 => [96.5217,110.086,447.113,218.324,251.909,544.491,874.477,433.007,1447.24…
  3 => [95.9246,110.25,447.255,217.838,251.194,544.302,874.151,431.784,1445.79,…
  5 => [94.4035,110.42,447.359,216.541,249.5,543.658,873.237,429.019,1442.56,51…
  8 => [79.231,114.912,453.665,214.252,232.39,537.505,864.907,414.169,1422.04,5…
  6 => [87.5052,111.69,448.88,212.471,241.735,540.975,869.226,418.57,1430.08,51…
  1 => [98.7226,108.564,444.608,220.179,255.035,544.795,875.241,440.222,1456.21…

In [18]:
norObjFunDict

Dict{Any,Any} with 9 entries:
  7 => 0.363959
  4 => 0.686442
  9 => 0.275342
  2 => 0.709884
  3 => 0.694766
  5 => 0.637127
  8 => 0.275342
  6 => 0.461158
  1 => 1.0

In [19]:
coeffs_dict

Dict{Any,Any} with 9 entries:
  (6,3.5,1.0,1) => [1.0,0.0516668,-0.164568,0.111851,0.137789,-0.0102345,0.0020…
  (6,3.5,1.0,8) => [1.0,0.010683,-0.0596691,0.086489,0.103029,0.00825354,0.0002…
  (6,3.5,1.0,2) => [1.0,0.0293022,-0.122934,0.125963,0.102071,0.003343,0.000976…
  (6,3.5,1.0,7) => [1.0,-0.0202654,0.0310233,0.0864107,0.0588593,0.0172204,0.00…
  (6,3.5,1.0,4) => [1.0,0.0292887,-0.122879,0.125923,0.101946,0.00346002,0.0009…
  (6,3.5,1.0,5) => [1.0,0.0291864,-0.12242,0.125343,0.101599,0.00396457,0.00082…
  (6,3.5,1.0,3) => [1.0,0.029288,-0.122875,0.125914,0.101956,0.00345521,0.00095…
  (6,3.5,1.0,9) => [1.0,0.010683,-0.0596691,0.086489,0.103029,0.00825354,0.0002…
  (6,3.5,1.0,6) => [1.0,0.028781,-0.120163,0.12007,0.106664,0.00188979,0.001124…

In [20]:
outfile = open("./results/norObjFunDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, norObjFunDict)

close(outfile)

In [21]:
outfile = open("./results/demandsDiffDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDiffDict)

close(outfile)

In [22]:
outfile = open("./results/objInvVIDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, objInvVIDict)

close(outfile)

In [23]:
outfile = open("./results/coeffs_dict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)

In [24]:
outfile = open("./results/demandsVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsVecDict)

close(outfile)

In [25]:
outfile = open("./results/demandsDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDict)

close(outfile)

In [26]:
outfile = open("./results/tapFlowDicDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [27]:
outfile = open("./results/tapFlowVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowVecDict)

close(outfile)